[Kuber.jl](https://github.com/JuliaComputing/Kuber.jl) is a Julia Kubernetes client.

The package can be fetched with a Pkg.add. If already installed, we can simply start `using` it.

In [1]:
# using Pkg
# Pkg.add("Kuber")
using Kuber

## Connecting, Authenticating, API versions

To connect with the cluster, we create a kubernetes context.

Here we are connecting via a local proxy, called `kubectl proxy`, which is convenient because it takes up the authentication part and gives a local port to connect to. The `kubectl proxy` can be used with all types of clusters, even remote cloud clusters.

In [2]:
ctx = KuberContext()

Kubernetes namespace default at http://localhost:8001

It is also possible to specify authentication parameters while constructing a KuberContext. We shall not get into the details of that here. But they will essentially set these parameters - the namespace, the URL of the API server to connect to, and the HTTP options - including the HTTP headers and certificates that can be used to authenticate.

**APIs available for setting namespaces and advanced authentication:**
- `set_ns(ctx::KuberContext, namespace::String)`
- `set_server(ctx::KuberContext, uri::String=DEFAULT_URI, reset_api_versions::Bool=false; kwargs...)`

In [3]:
ctx.namespace

"default"

In [4]:
ctx.client.root

"http://localhost:8001"

In [5]:
ctx.client.clntoptions

Dict{Symbol,Any} with 3 entries:
  :require_ssl_verification => true
  :retries                  => 0
  :status_exception         => false

In [6]:
ctx.client.headers

Dict{String,String} with 1 entry:
  "Connection" => "close"

Once connected, the next step is to discover the APIs the cluster supports. It is not required for users to call this method. It is done here for demonstration, otherwise this will be done automatically on first use of any API.

Clusers may support only specific APIs and specific versions of those. Kuber.jl fetches that information and auto-configures itself.

In [7]:
Kuber.set_api_versions!(ctx; verbose=true)

┌ Info: Core versions
│   supported = v1
│   preferred = v1
└ @ Kuber /home/tan/.julia/packages/Kuber/fdQId/src/helpers.jl:158
┌ Info: apiregistration.k8s.io (Apiregistration) versions
│   supported = v1, v1beta1
│   preferred = v1
└ @ Kuber /home/tan/.julia/packages/Kuber/fdQId/src/helpers.jl:125
┌ Info: extensions (Extensions) versions
│   supported = v1beta1
│   preferred = v1beta1
└ @ Kuber /home/tan/.julia/packages/Kuber/fdQId/src/helpers.jl:125
┌ Info: apps (Apps) versions
│   supported = v1
│   preferred = v1
└ @ Kuber /home/tan/.julia/packages/Kuber/fdQId/src/helpers.jl:125
┌ Info: events.k8s.io (Events) versions
│   supported = v1beta1
│   preferred = v1beta1
└ @ Kuber /home/tan/.julia/packages/Kuber/fdQId/src/helpers.jl:125
┌ Info: authentication.k8s.io (Authentication) versions
│   supported = v1, v1beta1
│   preferred = v1
└ @ Kuber /home/tan/.julia/packages/Kuber/fdQId/src/helpers.jl:125
┌ Info: authorization.k8s.io (Authorization) versions
│   supported = v1, v1beta1
│   

## REST API paradigm - entities and verbs


Entities:
- Job
- Pod
- Service

Verbs:
- get/list
- put
- update!
- delete!

## Examining the cluster

As an example, we are here getting the list of components of the cluster itself and their statuses, by doing a `get` on the `ComponenetStatus` entity.

What we get back is a `ComponentStatusList` object. And what is of interest to us in that is this element named `items`, which is an array of `ComponentStatus`.

In [8]:
result = get(ctx, :ComponentStatus);
typeof(result)

Kuber.Kubernetes.IoK8sApiCoreV1ComponentStatusList

In [9]:
result

{
  "kind": "ComponentStatusList",
  "metadata": {
    "selfLink": "/api/v1/componentstatuses"
  },
  "apiVersion": "v1",
  "items": [
    {
      "metadata": {
        "selfLink": "/api/v1/componentstatuses/scheduler",
        "name": "scheduler"
      },
      "conditions": [
        {
          "status": "True",
          "type": "Healthy",
          "message": "ok"
        }
      ]
    },
    {
      "metadata": {
        "selfLink": "/api/v1/componentstatuses/controller-manager",
        "name": "controller-manager"
      },
      "conditions": [
        {
          "status": "True",
          "type": "Healthy",
          "message": "ok"
        }
      ]
    },
    {
      "metadata": {
        "selfLink": "/api/v1/componentstatuses/etcd-0",
        "name": "etcd-0"
      },
      "conditions": [
        {
          "status": "True",
          "type": "Healthy",
          "message": "{\"health\":\"true\"}"
        }
      ]
    }
  ]
}


From what we got, we can now print out names of continers like this, for example.

In [10]:
collect(item.metadata.name for item in result.items)

3-element Array{String,1}:
 "scheduler"
 "controller-manager"
 "etcd-0"

In the same manner we can get other properties of the cluster as well. Here we are listing the namespaces. Namespaces are like separate clusters within the cluster, like having a separate account for each department or each person.

In [11]:
collect(item.metadata.name for item in (get(ctx, :Namespace)).items)

6-element Array{String,1}:
 "default"
 "juliarun"
 "kube-node-lease"
 "kube-public"
 "kube-system"
 "kubernetes-dashboard"

We were in the default namespace if you remember. If we list processes, which are entities called `Pods` in Kubernetes, we find none.

Because we haven't started any. But if we switch to the system namespace, we shall find some already running. These are what Kubernetes itself started - the system processes.

In [12]:
collect(item.metadata.name for item in (get(ctx, :Pod)).items)

0-element Array{Any,1}

In [13]:
set_ns(ctx, "kube-system")

"kube-system"

In [14]:
collect(item.metadata.name for item in (get(ctx, :Pod)).items)

10-element Array{String,1}:
 "calico-kube-controllers-75d56dfc47-5cjkv"
 "calico-node-l2294"
 "coredns-66bff467f8-p9g2q"
 "coredns-66bff467f8-prj5c"
 "etcd-tanlto"
 "kube-apiserver-tanlto"
 "kube-controller-manager-tanlto"
 "kube-proxy-bpg64"
 "kube-scheduler-tanlto"
 "npd-v0.8.0-6fdv9"

What kubernetes runs, the `Pods` in the cluster, are containers. And containers are instances of packaged applications. The packages are called images. The images are stored in a place called repository. We can see images in our repository.

In [17]:
run(pipeline(`docker images`, `grep -e nginx -e openresty -e juliadefault -e rabbitmq`));

openresty                            bionic-with-top     a82cdeb5f7e6        2 days ago          648MB
tanlto:5000/juliadefault             v0.4.0              da283f42408e        7 days ago          431MB
127.0.1.1:5000/juliadefault          v0.4.0              da283f42408e        7 days ago          431MB
juliadefault                         v0.4.0              da283f42408e        7 days ago          431MB
127.0.1.1:5000/openresty/openresty   bionic              de816c768659        2 weeks ago         562MB
openresty/openresty                  bionic              de816c768659        2 weeks ago         562MB
127.0.1.1:5000/rabbitmq              3.8.2-management    985adbf13062        2 months ago        181MB
rabbitmq                             3.8.2-management    985adbf13062        2 months ago        181MB
tanlto:5000/rabbitmq                 3.8.2-management    985adbf13062        2 months ago        181MB
127.0.1.1:5000/rabbitmq              3.8.2               b8956a8129ef    

Images can be fetched from a central repository using `docker pull <image name>`

For example, `docker pull openresty:bionic` would fetch the `openresty/openresty:bionic` image from dockerhub. Dockerhub is a hosted repository at https://hub.docker.com/search?q=&type=image

Applications can be built as images using the `docker build` command. For example the `openresty:bionic-with-top` was an image built locally. The specification file for that, called a dockerfile, looks like this:

In [18]:
;cat Dockerfile

# openresty image with ngxtop utility
#
# build using:
# docker build -t openresty:bionic-with-top -f Dockerfile .

FROM openresty/openresty:bionic

RUN apt-get install -y python-pip \
    && pip install ngxtop \
    && rm /usr/local/openresty/nginx/logs/access.log \
    && rm /usr/local/openresty/nginx/logs/error.log

COPY nginx.conf /usr/local/openresty/nginx/conf/nginx.conf
